# FAISS (Facebook AI Similarity Search)
- https://github.com/facebookresearch/faiss
- https://faiss.ai/
- 应用场景：基于 embedding 找到 nearest neighbor

In [1]:
import numpy as np
import faiss
import requests
from io import StringIO
import pandas as pd

## 获取数据集

In [2]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')

text = res.text
text[:100]

'pair_ID\tsentence_A\tsentence_B\trelatedness_score\tentailment_judgment\n1\tA group of kids is playing in '

转换成 DataFrame

In [3]:
data = pd.read_csv(StringIO(text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


准备数据

In [4]:
sentences = data['sentence_A'].tolist()

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [5]:
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)

sentences = list(set(sentences[0:1000]))
len(set(sentences))

718

In [6]:
sentences[:10]

['The man is carelessly smearing butter on a slice of garlic bread',
 'A woman is sewing with a machine',
 'A black helmet is being worn by a man who is pushing a bicycle',
 'A white and brown dog is walking through the water with difficulty',
 'Two toddlers are eating on a really small wagon full of corndogs ',
 'There is no man walking outside',
 'A woman is cutting a white onion',
 'A deer is jumping over a fence',
 'The cat is licking a shallow dish full of milk',
 'Nobody is riding the bicycle on one wheel']

## 生成句向量
- https://www.sbert.net/
- [Sentence-BERT](https://arxiv.org/pdf/1908.10084.pdf)

In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [8]:
sentence_embeddings = model.encode(sentences)

In [9]:
sentence_embeddings.shape

(718, 768)

## 存入 Faiss

In [10]:
d = sentence_embeddings.shape[1]
d

768

### [1] Flat L2 Index (exhaustive search, 准确但速度慢)

In [11]:
# https://github.com/facebookresearch/faiss/wiki/Faiss-indexes
# Flat indexes just encode the vectors into codes of a fixed size and store them in an array of ntotal * code_size bytes.
# At search time, all the indexed vectors are decoded sequentially and compared to the query vectors.
index = faiss.IndexFlatL2(d)

In [12]:
index.is_trained

True

In [13]:
# 通过 add 方法加入向量
index.add(sentence_embeddings)

In [14]:
index.ntotal

718

开始查询

In [15]:
k = 4
query = model.encode(["Someone is playing the football"])

In [16]:
query

array([[ 2.83163905e-01,  1.57802433e-01,  5.15145421e-01,
         1.29240781e-01, -5.52885413e-01,  1.25420973e-01,
         8.56239572e-02,  2.13250238e-02, -1.03120327e+00,
        -4.62618887e-01, -9.21380103e-01,  4.73979712e-01,
        -1.93609335e-02,  4.93645042e-01,  1.18129420e+00,
         1.72137365e-01,  1.56022355e-01, -1.19113266e+00,
         2.44104117e-01,  7.24390149e-02, -8.05312037e-01,
        -5.48978031e-01, -5.80017269e-01, -7.24116504e-01,
         2.25761145e-01,  2.23430082e-01,  4.72022802e-01,
         4.48654979e-01, -9.23286915e-01,  8.72921586e-01,
         3.72225940e-01,  3.36247236e-01,  2.17777252e-01,
         1.16004169e-01, -7.84430087e-01, -4.05083686e-01,
         3.56108636e-01, -6.44741058e-01,  4.10916477e-01,
         2.63744384e-01, -3.26047897e-01,  3.29789013e-01,
        -8.08365643e-01,  6.68196678e-02, -8.74249101e-01,
        -1.16700292e+00, -6.89029217e-01,  3.20187747e-01,
        -1.07627809e+00, -3.70335549e-01,  3.77273828e-0

In [17]:
%%time
D, I = index.search(query, k)  

CPU times: user 6 ms, sys: 7.1 ms, total: 13.1 ms
Wall time: 15.3 ms


In [18]:
D

array([[ 30.626198, 199.8423  , 200.01804 , 206.05267 ]], dtype=float32)

In [19]:
I

array([[320, 500, 476,  49]])

查看相似句子

In [20]:
[f'{i}: {sentences[i]}' for i in I[0]]

['320: Two groups of people are playing football',
 '500: The little kid is playing football and falling harmlessly into the grass',
 '476: A player is running with the ball',
 '49: A man who is playing is running with the ball in his hands']

通过 reconstruct 获取 vector

In [21]:
vecs = np.zeros((k, d))
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

In [22]:
vecs.shape

(4, 768)

In [23]:
vecs[3][:10]

array([ 0.49663264,  0.00587879,  0.60509884,  0.40411866, -0.32720342,
       -0.65628713, -0.60181159, -0.13350478, -0.72880864, -0.32354006])

In [24]:
sentence_embeddings[49][0:10]

array([ 0.49663264,  0.00587879,  0.60509884,  0.40411866, -0.32720342,
       -0.65628713, -0.6018116 , -0.13350478, -0.72880864, -0.32354006],
      dtype=float32)

## [2] Flat inner product / cosine similarity Index

In [25]:
# https://medium.datadriveninvestor.com/cosine-similarity-cosine-distance-6571387f9bf8
ip_index = faiss.IndexFlatIP(d)
ip_index.add(sentence_embeddings)
D, I = ip_index.search(query, k)
D

array([[281.67133, 196.91997, 193.19354, 192.44211]], dtype=float32)

In [26]:
I

array([[320, 476,  49, 500]])

In [27]:
# normalize_L2 例子
a = np.array([[1, 1]]).astype(np.float32)
faiss.normalize_L2(a)
a

array([[0.70710677, 0.70710677]], dtype=float32)

In [28]:
# https://github.com/facebookresearch/faiss/issues/95
import copy
cosine_index = faiss.IndexFlatIP(d)
sentence_embeddings_copy = copy.deepcopy(sentence_embeddings)
faiss.normalize_L2(sentence_embeddings_copy)
cosine_index.add(sentence_embeddings_copy)
query_copy = copy.deepcopy(query)
faiss.normalize_L2(query_copy)

In [29]:
D, I = cosine_index.search(query_copy, k)
D

array([[0.9484966, 0.6632283, 0.6582364, 0.6522202]], dtype=float32)

In [30]:
I

array([[320, 476, 500,  49]])

## [3] 基于 partition 的加速搜索 `IndexIVFFlat`
- IVF: Inverted file
- [Voronoi cells](https://en.wikipedia.org/wiki/Voronoi_diagram) 
- 搜索过程：首先找到 query 所在的 cell，利用 quantizer 计算 query 和 cell 内部的 vector 距离
- https://mccormickml.com/2017/10/22/product-quantizer-tutorial-part-2/

In [48]:
# nlist: 多少个 partitions
nlist = 50
# 距离度量方式
quantizer = faiss.IndexFlatL2(d)
ivf_index = faiss.IndexIVFFlat(quantizer, d, nlist)
ivf_index.nprobe

1

In [49]:
# 该 index 要求首先对数据做预处理（即划分 partition）
ivf_index.is_trained

False

In [50]:
# 开始训练
ivf_index.train(sentence_embeddings)
ivf_index.is_trained

True

In [51]:
ivf_index.add(sentence_embeddings)
ivf_index.ntotal

718

查看相似句子

In [52]:
%%time
D, I = ivf_index.search(query, k)
print(I)

[[320  14  87 616]]
CPU times: user 2.47 ms, sys: 908 µs, total: 3.38 ms
Wall time: 677 µs


为了增加搜索范围（即多个 cell），可以增大 `nprobe` 的值。但是会增加搜索时间。

In [54]:
ivf_index.nprobe = 10

In [56]:
%%time
D, I = ivf_index.search(query, k)
print(I)

[[320 500 476  49]]
CPU times: user 2.99 ms, sys: 1.06 ms, total: 4.05 ms
Wall time: 923 µs


在调用 `reconstruct` 方法前，需要调用 `make_direct_map` 方法

In [59]:
ivf_index.make_direct_map()

In [60]:
ivf_index.reconstruct(49)[:10]

array([ 0.49663264,  0.00587879,  0.60509884,  0.40411866, -0.32720342,
       -0.65628713, -0.6018116 , -0.13350478, -0.72880864, -0.32354006],
      dtype=float32)

## 基于压缩 (Quantization) 的加速搜索 PQ (Product Quantization)
- `Flat` 没有对输入 vector 做任何处理，当数据量较大时，存储 full vector 占用较大存储空间。Faiss 基于 *Product Quantization* (PQ) 可以对 vector 进行压缩。与 **IVF** 一样，都是降低搜索空间的 approximate 方法。
- PQ 分为三步：
    - 将原始 vector 划分成 多个 subvectors
    - 对 subvectors 集合做聚类，找到类 centroids
    - 将原始 vector 的多个 subvectors 使用 subvector 最近的 centroid ID 代替
- https://mccormickml.com/2017/10/13/product-quantizer-tutorial-part-1/

In [61]:
# 划分成多少个 sub vector，需要被 d 整除
m = 8 
# 使用 8 bits 表示每个 centroid
bits = 8

quantizer = faiss.IndexFlatL2(d) 
pq_index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits) 

In [62]:
pq_index.is_trained

False

In [64]:
pq_index.train(sentence_embeddings)
pq_index.is_trained

True

In [65]:
pq_index.add(sentence_embeddings)

In [66]:
pq_index.nprobe = 10

In [71]:
%%time
D, I = pq_index.search(query, k)
print(I)

[[320 476  49 500]]
CPU times: user 2.33 ms, sys: 945 µs, total: 3.27 ms
Wall time: 991 µs


注：运算时间会有波动，当数据量大时趋于稳定。